# Outlier Detection for Time Series with Recurrent Autoencoder Ensembles


## Imports

In [ ]:
# A package for multi input - multi output random search
!pip install keras-hypetune

In [ ]:
import os
import random
from time import time
from google.colab import drive
from functools import partial
from joblib import Parallel, delayed

import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm
from scipy import stats

from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tensorflow import keras
from tensorflow.keras import Model, Input, layers

from tensorflow.keras.layers import Dense, Bidirectional, LSTM, LSTMCell, GRU, GRUCell, Reshape, Dropout, GaussianNoise, Concatenate, Lambda, RepeatVector, TimeDistributed

from kerashypetune import KerasRandomSearchCV

In [ ]:
drive.mount('/content/drive', force_remount=True)

## Data Utilities

In [ ]:
TIME_STEPS = 6

# Generated training sequences for use in the model.
def create_sequences(values, time_steps=TIME_STEPS, stride=1):
    output = []
    for i in range(0, len(values) - time_steps, stride):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

## Evaluation Utilities

In [ ]:
def _enumerate_thresholds(rec_errors, n=1000):
    # maximum value of the anomaly score for all time steps in the test data
    thresholds, step_size = [], np.max(rec_errors) / n
    th = 0.
    # create a uniform threshold values
    for i in range(n):
        thresholds.append(th)
        th = th + step_size
    
    return thresholds

In [ ]:
def _compute_anomaly_scores(x, rec_x, x_val=None, scoring='square_median'):
    if scoring == 'absolute':
        return np.mean(np.abs(rec_x - x), axis=-1)
    elif scoring == 'square_mean':
        return np.mean(np.square(rec_x - x), axis=-1)
    elif scoring == 'square_median':
        # used in the paper
        return np.median(np.square(rec_x - x), axis=-1)

In [ ]:
# compute the metrics for the model predictoin
def evaluate(x, rec_x, labels, is_reconstructed=True, n=1000, scoring='square_median', x_val=None):
    TP, TN, FP, FN = [], [], [], []
    precision, f1, tpr, fpr = [], [], [], []
    
    rec_errors = _compute_anomaly_scores(x, rec_x, scoring) if is_reconstructed else rec_x
    if len(rec_errors.shape) > 2:
        if scoring.split('_')[1] == 'mean':
            rec_errors = np.mean(rec_errors, axis=0)
        else:
            rec_errors = np.median(rec_errors, axis=0)

    # get uniform threshold values    
    thresholds = _enumerate_thresholds(rec_errors, n)
    
    for th in thresholds: # for each threshold
        TP_t, TN_t, FP_t, FN_t = 0, 0, 0, 0
        for t in range(len(x)): # for each time window
            # if any part of the segment has an anomaly, we consider it as anomalous sequence
            true_anomalies, pred_anomalies = set(np.where(labels[t] == 1)[0]), set(np.where(rec_errors[t] > th)[0])
            if len(pred_anomalies) > 0 and len(pred_anomalies.intersection(true_anomalies)) > 0:
                # correct prediction (at least partial overlap with true anomalies)
                TP_t = TP_t + 1
            elif len(pred_anomalies) == 0 and len(true_anomalies) == 0:
                # correct rejection, no predicted anomaly on no true labels
                TN_t = TN_t + 1 
            elif len(pred_anomalies) > 0 and len(true_anomalies) == 0:
                # false alarm (i.e., predict anomalies on no true labels)
                FP_t = FP_t + 1
            elif len(pred_anomalies) == 0 and len(true_anomalies) > 0:
                # predict no anomaly when there is at least one true anomaly within the seq.
                FN_t = FN_t + 1
        
        TP.append(TP_t)
        TN.append(TN_t)
        FP.append(FP_t)
        FN.append(FN_t)
    
    for i in range(len(thresholds)):
        precision.append(TP[i] / (TP[i] + FP[i] + 0.0000001))
        tpr.append(TP[i] / (TP[i] + FN[i] + 0.0000001)) # recall or true positive rate (TPR)
        fpr.append(FP[i] / (FP[i] + TN[i] + 0.0000001))
        f1.append(2 * (precision[i] * tpr[i]) / (precision[i] + tpr[i] + 0.0000001))
    
    return {
        'rec_errors': rec_errors,
        'precision': np.mean(precision),
        'tpr': np.mean(tpr),
        'fpr': np.mean(fpr),
        'f1': np.mean(f1),
        'pr_auc': auc(tpr, precision),
        'roc_auc': auc(fpr, tpr),
    }

In [ ]:
def calculate_accuracy(test_y, rec_errors, threshold=0.5):
    prediction, ground_truth = [], []
    for t in range(len(test_y)): # for each time window
            true_anomalies, pred_anomalies = set(np.where(test_y[t] == 1)[0]), set(np.where(rec_errors[t] > threshold)[0])
            if len(pred_anomalies) > 0 and len(pred_anomalies.intersection(true_anomalies)) > 0:
                prediction.append(1)
                ground_truth.append(1)
            elif len(pred_anomalies) == 0 and len(true_anomalies) == 0:
                prediction.append(0)
                ground_truth.append(0)
            elif len(pred_anomalies) > 0 and len(true_anomalies) == 0:
                prediction.append(1)
                ground_truth.append(0)
            elif len(pred_anomalies) == 0 and len(true_anomalies) > 0:
                prediction.append(0)
                ground_truth.append(1)
    return accuracy_score(ground_truth, prediction)

## Models

### LSTM AE - Baseline Model

In [ ]:
def LSTM_AE(X_train,  param):
    LSTM = layers.LSTM
    model = keras.Sequential(
        [
            layers.InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])),
            LSTM(64, return_sequences=True),
            LSTM(32),
            layers.RepeatVector(X_train.shape[1]),
            LSTM(32, return_sequences=True),
            LSTM(64),
            layers.Dense(X_train.shape[1] *  X_train.shape[2]),
            layers.Reshape([X_train.shape[1], X_train.shape[2]])
        ]
    )
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=param['learning_rate'], clipnorm=param['clipnorm']), loss=param['loss'])
    return model

### S-RNN - Paper's Model


In [ ]:
# baisc S-RNN unit
class SkipRNN(tf.keras.layers.Layer):
    def __init__(self, cell, return_sequences=False, **kwargs):
        super().__init__(**kwargs)
        self.cell = cell
        self.return_sequences = return_sequences
        self.get_initial_state = getattr(
            self.cell, "get_initial_state", self.fallback_initial_state)
    def fallback_initial_state(self, inputs):
        return [tf.zeros([self.cell.state_size], dtype=inputs.dtype)]
    @tf.function
    def call(self, inputs, states=None):
        states = self.get_initial_state(inputs) if states == None else states

        outputs = tf.zeros(shape=[self.cell.output_size], dtype=inputs.dtype)
        outputs, states = self.cell(inputs, states)

        return outputs, states

In [ ]:
def S_RNN(params):
    tf.keras.backend.clear_session()

    # weight vector - as described in the paper
    sparseness_weights = [(0, 1), (1, 0), (1, 1)]
    N, N_LAYERS, N_UNITS = 10, 1, 8
  
    seq_length, dim = params['seq_length'], params['dim']

    en_input = Input(shape=[seq_length, dim])
    X = GaussianNoise(0.5)(en_input) # add noise to input

    # create encoder part and the shared vecotr of the ensemble
    shared_latents = []
    for i in range(N):
        prev_states = []
        skip_length = np.random.randint(low=2, high=params['high'], size=1)[0] # random skip size selection
        w1, w2 = np.array(sparseness_weights)[np.random.choice(3, size=1)][0] # random weight vector selection
        w = w1 + w2

        for t in range(seq_length):
            Xt = Lambda(lambda x: x[:, t, :])(X)
            if t == 0:
                O, H = SkipRNN(GRUCell(N_UNITS))(Xt)
            else:
                if t - skip_length >= 0:
                    # apply skip connection
                    states = (w1 * prev_states[t-1] + w2 * prev_states[t-skip_length]) / w
                    O, H = SkipRNN(GRUCell(N_UNITS))(Xt, states)
                else:
                    O, H = SkipRNN(GRUCell(N_UNITS))(Xt, prev_states[t-1])

            prev_states.append(H)
        shared_latents.append(H)

    de_outputs = []
    de_input = Concatenate()(shared_latents)
    D = Dense(dim, kernel_regularizer=tf.keras.regularizers.l1(params['l1']))(de_input)

    # create decoder part of the ensemble
    for i in range(N):
        Y_i = []
        prev_states = []
        skip_length = np.random.randint(low=2, high=params['high'], size=1)[0]
        w1, w2 = np.array(sparseness_weights)[np.random.choice(3, size=1)][0]
        w = w1 + w2

        for t in range(seq_length):
            if t == 0:
                y = Dense(dim)(D)
                _, H = SkipRNN(GRUCell(dim))(y, D) # y_t
            else:
                if t - skip_length >= 0:
                    # apply skip connection
                    states = (w1 * prev_states[t-1] + w2 * prev_states[t-skip_length]) / w
                    y, H = SkipRNN(GRUCell(dim))(Y_i[t-1], states)
                else:
                    y, H = SkipRNN(GRUCell(dim))(Y_i[t-1], prev_states[t-1])

            Y_i.append(y)
            prev_states.append(H)

        # current level autoencoder output
        Y_i = Concatenate()(Y_i)
        Y_i = Reshape([seq_length, dim])(Y_i) # resize to match original dimensions
        de_outputs.append(Y_i)

    model = Model(inputs=en_input, outputs=de_outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
    
    return model

### LIS-RNN - Linear Incremental S-RNN - Our Improved Version

In [ ]:
def LIS_RNN(params):
    tf.keras.backend.clear_session()

    # weight vector as described in the paper
    sparseness_weights = [(0, 1), (1, 0), (1, 1)]
    N, N_LAYERS, N_UNITS = 10, 1, 8

    seq_length, dim = params['seq_length'], params['dim']

    en_input = Input(shape=[seq_length, dim])
    X = GaussianNoise(0.5)(en_input)

    shared_latents = []
    for i in range(N):
        prev_states = []
        skip_length = i+1 # linear skip size selection
        w1, w2 = np.array(sparseness_weights)[np.random.choice(3, size=1)][0]
        w = w1 + w2

        for t in range(seq_length):
            Xt = Lambda(lambda x: x[:, t, :])(X)
            if t == 0:
                O, H = SkipRNN(GRUCell(N_UNITS))(Xt)
            else:
                if t - skip_length >= 0:
                    states = (w1 * prev_states[t-1] + w2 * prev_states[t-skip_length]) / w
                    O, H = SkipRNN(GRUCell(N_UNITS))(Xt, states)
                else:
                    O, H = SkipRNN(GRUCell(N_UNITS))(Xt, prev_states[t-1])

            prev_states.append(H)
        shared_latents.append(H)

    de_outputs = []
    de_input = Concatenate()(shared_latents)
    # global shared representation
    D_shared = Dense(dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(params['l1']))(de_input)

    for i in range(N):
        Y_i = []
        prev_states = []
        skip_length = i+1 # linear skip size selection
        w1, w2 = np.array(sparseness_weights)[np.random.choice(3, size=1)][0]
        w = w1 + w2
        
        # global + specific representation for each level
        D_each = Dense(dim, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(params['l1']))(shared_latents[i])

        D = Concatenate()([D_shared, D_each])
        D = Dense(dim)(D)

        for t in range(seq_length):
            if t == 0:
                y = Dense(dim)(D)
                _, H = SkipRNN(GRUCell(dim))(y, D)
            else:
                if t - skip_length >= 0:
                    states = (w1 * prev_states[t-1] + w2 * prev_states[t-skip_length]) / w
                    y, H = SkipRNN(GRUCell(dim))(Y_i[t-1], states)
                else:
                    y, H = SkipRNN(GRUCell(dim))(Y_i[t-1], prev_states[t-1])

            Y_i.append(y)
            prev_states.append(H)

        # current level autoencoder output
        Y_i = Concatenate()(Y_i)
        Y_i = Reshape([seq_length, dim])(Y_i)
        de_outputs.append(Y_i)

    model = Model(inputs=en_input, outputs=de_outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=params['loss'])
    
    return model

## Experiments

### LSTM-AE - Baseline Model 

In [ ]:
def process_LSTM(name, data_df,label_df, seq_length = 8, stride = 4): 
    total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': [], 'tpr':[], 'fpr':[], 'precision':[], 'accuracy':[], 'training_time(sec)':[], 'Inference_time(sec)':[],'folds_best_params':[]}
    kfold = KFold(10, shuffle=True)
    for i, (train, test) in enumerate(kfold.split(data_df)):
        t1=time()
        
        train_df = data_df.iloc[train]
        test_df = data_df.iloc[test]
        train_labels = label_df.iloc[train].values
        labels = label_df.iloc[test].values
        
        scaler = MinMaxScaler()
        train_df = scaler.fit_transform(train_df)
        test_df = scaler.transform(test_df)
        
        # creating sequence train and test data
        X_train = create_sequences(train_df, time_steps=seq_length, stride=stride)
        X_test = create_sequences(test_df, time_steps=seq_length, stride=stride)
        
        y_train = create_sequences(train_labels, time_steps=seq_length, stride=stride)
        y_test = create_sequences(labels, time_steps=seq_length, stride=stride)
        
        # hyperparameters for the model
        param_grid = {
            'learning_rate': [0.001, 0.002,0.005, 0.006, 0.007, 0.01, 0.015],
            'loss': ['mse','msle','mae','mape',],
            'clipnorm': [1,2,2.5]
        }
        t = time()
        cv = KFold(n_splits=3)

        # perform random search
        krs = KerasRandomSearchCV(partial(LSTM_AE, X_train), param_grid, cv=cv, n_iter=50, sampling_seed=33,
                          monitor='val_loss', greater_is_better=False, tuner_verbose=0)

        krs.search(X_train, X_train, callbacks=[es], epochs=1)
        training_time = round(time()-t,4)

        # get best hyperparameters
        folds_best_scores = krs.folds_best_score 
        best_fold_index = min(folds_best_scores, key=folds_best_scores.get)
        best_model = krs.folds_best_models[best_fold_index] 
        folds_best_params = krs.folds_best_params[best_fold_index] 
        
        # inference stage
        test_sample = data_df.sample(1000, replace=True)
        test_sample = create_sequences(test_sample, time_steps=seq_length, stride=stride)
        t = time()
        best_model.predict(test_sample)
        inference_time = round(time()-t,4)

        scores = evaluate(X_test, best_model.predict(X_test),y_test , is_reconstructed=True)# scoring='square_median'

        # save current fold results
        total_scores['dataset'].append(name)
        total_scores['f1'].append(scores['f1'])
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        total_scores['fpr'].append(scores['fpr'])
        total_scores['tpr'].append(scores['tpr'])
        total_scores['precision'].append(scores['precision'])
        total_scores['accuracy'].append(calculate_accuracy(y_test, scores['rec_errors']))
        total_scores['training_time(sec)'].append(training_time)
        total_scores['Inference_time(sec)'].append(inference_time)
        total_scores['folds_best_params'].append(folds_best_params)
                
    return total_scores

### S-RNN - Paper's Model

In [ ]:
seq_length = 8
stride = 4
es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)#monitor="loss"

In [ ]:
def process_SRNN(name, data_df, label_df, seq_length = 8, stride = 4): 
    total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': [], 'tpr':[], 'fpr':[], 'precision':[], 'accuracy':[], 'training_time(sec)':[],  'Inference_time(sec)':[],'folds_best_params':[]}
    kfold = KFold(10, shuffle=True)
    for i, (train, test) in enumerate(kfold.split(data_df)):
        t1 = time()
        
        train_df = data_df.iloc[train]
        test_df = data_df.iloc[test]
        train_labels = label_df.iloc[train].values
        labels = label_df.iloc[test].values
        
        scaler = MinMaxScaler()
        train_df = scaler.fit_transform(train_df)
        test_df = scaler.transform(test_df)
        
        # creating sequence train and test data
        X_train = create_sequences(train_df, time_steps=seq_length, stride=stride)
        X_test = create_sequences(test_df, time_steps=seq_length, stride=stride)
        
        y_train = create_sequences(train_labels, time_steps=seq_length, stride=stride)
        y_test = create_sequences(labels, time_steps=seq_length, stride=stride)
        
        # hyperparameters for the model
        param_grid = {
                  'seq_length': X_train.shape[1], 
                  'dim': X_train.shape[2],
                  'high' : [5,7,8,9,10],
                  'l1': [0.001, 0.002,0.003,0.004,0.005, 0.006, 0.007, 0.008, 0.009, 0.01],
              }

        t = time()
        cv = KFold(n_splits=3)

        # perform random search hyperparameters tuning
        kgs = KerasRandomSearchCV(S_RNN, param_grid, cv=cv, n_iter=50, sampling_seed=33, monitor='val_loss', greater_is_better=False, tuner_verbose=0)
        kgs.search(X_train, [np.flip(X_train, axis=1) for _ in range(10)], callbacks=[es], epochs=1)
        
        training_time = round(time()-t,4)

        # get best model by score
        folds_best_scores = kgs.folds_best_score 
        best_fold_index = min(folds_best_scores, key=folds_best_scores.get)
        best_model = kgs.folds_best_models[best_fold_index] 
        folds_best_params = kgs.folds_best_params[best_fold_index] 

        # inference stage
        test_sample = data_df.sample(1000, replace=True)
        test_sample = create_sequences(test_sample, time_steps=seq_length, stride=stride)
        t = time()
        best_model.predict(test_sample)
        inference_time = round(time()-t,4)

        test_seq_rec = [np.flip(rec, axis=1) for rec in best_model.predict(X_test)]
        scores = evaluate(X_test, test_seq_rec, y_test, is_reconstructed=True,) #scoring='square_median')

        # save current fold results
        total_scores['dataset'].append(name)
        total_scores['f1'].append(scores['f1'])
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        total_scores['fpr'].append(scores['fpr'])
        total_scores['tpr'].append(scores['tpr'])
        total_scores['precision'].append(scores['precision'])
        total_scores['accuracy'].append(calculate_accuracy(y_test, scores['rec_errors']))
        total_scores['training_time(sec)'].append(training_time)
        total_scores['Inference_time(sec)'].append(inference_time)
        total_scores['folds_best_params'].append(folds_best_params)

    return total_scores

### LIS-RNN - Our Model

In [ ]:
seq_length = 8
stride = 4
es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min", restore_best_weights=True)#monitor="loss"

In [ ]:
# can be merged to one function with process_SRNN - just get the rs hypermodel as a parameter
def process_LISRNN(name, data_df, label_df, seq_length = 8, stride = 4): 
    total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': [], 'tpr':[], 'fpr':[], 'precision':[], 'accuracy':[], 'training_time(sec)':[],  'Inference_time(sec)':[],'folds_best_params':[]}
    kfold = KFold(10, shuffle=True)
    for i, (train, test) in enumerate(kfold.split(data_df)):
        print("Running fold " + str(i))
        t1 = time()
        
        train_df = data_df.iloc[train]
        test_df = data_df.iloc[test]
        train_labels = label_df.iloc[train].values
        labels = label_df.iloc[test].values
        
        scaler = MinMaxScaler()
        train_df = scaler.fit_transform(train_df)
        test_df = scaler.transform(test_df)
        
        # create data sequence for the model
        X_train = create_sequences(train_df, time_steps=seq_length, stride=stride)
        X_test = create_sequences(test_df, time_steps=seq_length, stride=stride)
        
        y_train = create_sequences(train_labels, time_steps=seq_length, stride=stride)
        y_test = create_sequences(labels, time_steps=seq_length, stride=stride)

        # hyperparameters for the model
        param_grid = {
                  'seq_length': X_train.shape[1], 
                  'dim': X_train.shape[2],
                  'loss': ['mse','msle','mae','mape',],
                  'l1': [0.001, 0.002, 0.003,0.004,0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.015, 0.02, 0.025],
              }

        t = time()
        cv = KFold(n_splits=3)
      
        # perform random search hyperparameters tuning
        kgs = KerasRandomSearchCV(LIS_RNN, param_grid, cv=cv, n_iter=5, sampling_seed=33, monitor='val_loss', greater_is_better=False, tuner_verbose=1)
        kgs.search(X_train, [np.flip(X_train, axis=1) for _ in range(10)], callbacks=[es], epochs=1)
        
        training_time = round(time()-t,2)

        # get best model by score
        folds_best_scores = kgs.folds_best_score 
        best_fold_index = min(folds_best_scores, key=folds_best_scores.get)
        best_model = kgs.folds_best_models[best_fold_index] 
        folds_best_params = kgs.folds_best_params[best_fold_index] 

        # inference stage
        test_sample = data_df.sample(1000, replace=True)
        test_sample = create_sequences(test_sample, time_steps=seq_length, stride=stride)
        t = time()
        best_model.predict(test_sample)
        inference_time = round(time()-t,4)

        test_seq_rec = [np.flip(rec, axis=1) for rec in best_model.predict(X_test)]
        scores = evaluate(X_test, test_seq_rec, y_test, is_reconstructed=True, scoring='square_median')

        # save current fold results
        total_scores['dataset'].append(name)
        total_scores['f1'].append(scores['f1'])
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        total_scores['fpr'].append(scores['fpr'])
        total_scores['tpr'].append(scores['tpr'])
        total_scores['precision'].append(scores['precision'])
        total_scores['accuracy'].append(calculate_accuracy(y_test, scores['rec_errors']))
        total_scores['training_time(sec)'].append(training_time)
        total_scores['Inference_time(sec)'].append(inference_time)
        total_scores['folds_best_params'].append(folds_best_params)

    return total_scores

# Run

For this section we have implemented to type of running methods. The first one, is a regular run, where the code is running sequentially.
The second one, is a parallel run, which allowed us to run multiple datasets at once.

In [ ]:
main_data_path = "/content/drive/MyDrive/ML_Final_Project/dataset_final/"

## regular


### LSTM run

In [ ]:
total_scores =[]

dirs = sorted([f for f in os.listdir(main_data_path) if not f.startswith('.')])

for dir_ in dirs:
    print(dir_)
    data_path = main_data_path + dir_
    datasets = sorted([f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))])
    for dataset in tqdm(datasets):
        print(dataset)

        if dir_ == 'yahoo' or dir_ == 'power':
            data_df = pd.read_csv(f'{data_path}/{dataset}')[['value']]
            label_df = pd.read_csv(f'{data_path}/{dataset}').iloc[:, -1]

        else:
            data_df = pd.read_csv(f'{data_path}/{dataset}').iloc[:, 0:-1].astype(float)   
            label_df = pd.read_csv(f'{data_path}/{dataset}').iloc[:, -1].astype(int)    


        if dir_ == 'ECG':
            total_scores.append(process_LSTM(dataset,data_df,label_df, seq_length = 32, stride = 16))
        else:
            total_scores.append(process_LSTM(dataset,data_df,label_df, seq_length = 16, stride = 4)) 


In [ ]:
res = pd.concat([pd.DataFrame(total_scores[i]) for i in range(len(total_scores))], axis=0)
res.to_csv('lstm.csv')

### SRNN Run

In [ ]:
total_scores =[]

dirs = sorted([f for f in os.listdir(main_data_path) if not f.startswith('.')])

for dir_ in dirs:
    print(dir_)
    data_path = main_data_path + dir_
    datasets = sorted([f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))])

    for dataset in tqdm(datasets):
        print(dataset)

        if dir_ == 'yahoo' or dir_ == 'power':
            data_df = pd.read_csv(f'{data_path}/{dataset}')[['value']]
            label_df = pd.read_csv(f'{data_path}/{dataset}').iloc[:, -1]

        else:
            data_df = pd.read_csv(f'{data_path}/{dataset}').iloc[:, 0:-1].astype(float)   
            label_df = pd.read_csv(f'{data_path}/{dataset}').iloc[:, -1].astype(int)    


        if dir_ == 'ECG':
            total_scores.append(process_SRNN(dataset,data_df,label_df, seq_length = 32, stride = 16))
        else:
            total_scores.append(process_SRNN(dataset,data_df,label_df, seq_length = 8, stride = 4)) 


In [ ]:
res = pd.concat([pd.DataFrame(total_scores[i]) for i in range(len(total_scores))], axis=0)
res.to_csv('srnn.csv')

### LIS-RNN Run

In [ ]:
total_scores =[]

dirs = sorted([f for f in os.listdir(main_data_path) if not f.startswith('.')])

for dir_ in dirs:
    print(dir_)
    data_path = main_data_path + dir_
    datasets = sorted([f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))])

    for dataset in tqdm(datasets):
        print(dataset)

        if dir_ == 'yahoo' or dir_ == 'power':
            data_df = pd.read_csv(f'{data_path}/{dataset}')[['value']]
            label_df = pd.read_csv(f'{data_path}/{dataset}').iloc[:, -1]

        else:
            data_df = pd.read_csv(f'{data_path}/{dataset}').iloc[:, 0:-1].astype(float)   
            label_df = pd.read_csv(f'{data_path}/{dataset}').iloc[:, -1].astype(int)    


        if dir_ == 'ECG':
            total_scores.append(process_LISRNN(dataset,data_df,label_df, seq_length = 8, stride = 4))
        else:
            total_scores.append(process_LISRNN(dataset,data_df,label_df, seq_length = 8, stride = 4)) 


In [ ]:
res = pd.concat([pd.DataFrame(total_scores[i]) for i in range(len(total_scores))], axis=0)
res.to_csv('lisrnn.csv')

## parallel

### parallel_LSTM

In [ ]:
def parallel_LSTM(path): 
    dir_ = path.split('/')[-2]
    if dir_ == 'yahoo' or dir_ == 'power':
        data_df = pd.read_csv(path)[['value']]
        label_df = pd.read_csv(path).iloc[:, -1]

    else:
        data_df = pd.read_csv(path).iloc[:, 0:-1].astype(float)   
        label_df = pd.read_csv(path).iloc[:, -1].astype(int)    


    if dir_ == 'ECG':
        return process_LSTM(path.split('/')[-1],data_df,label_df, seq_length = 32, stride = 16)
    else:
        return process_LSTM(path.split('/')[-1],data_df,label_df, seq_length = 16, stride = 4)
    

In [ ]:
t = time()
files = sorted([root+'/'+files for root,_,f in os.walk(main_data_path)  for files in f ])
total_scores_all = Parallel(n_jobs=20)(delayed(parallel_LSTM)(path) for path in files)
print(time()-t)

In [ ]:
res = pd.concat([pd.DataFrame(total_scores_all[i]) for i in range(len(total_scores_all))], axis=0)
res.to_csv('lstm.csv')
# res.groupby('dataset').mean()

### Parallel S-RNN

In [ ]:
def parallel_SRNN(path): 
    dir_ = path.split('/')[-2]
    if dir_ == 'yahoo' or dir_ == 'power':
        data_df = pd.read_csv(path)[['value']]
        label_df = pd.read_csv(path).iloc[:, -1]

    else:
        data_df = pd.read_csv(path).iloc[:, 0:-1].astype(float)   
        label_df = pd.read_csv(path).iloc[:, -1].astype(int)    


    if dir_ == 'ECG':
        return process_SRNN(path.split('/')[-1],data_df,label_df, seq_length = 32, stride = 16)
    else:
        return process_SRNN(path.split('/')[-1],data_df,label_df, seq_length = 8, stride = 4)
    

In [ ]:
t = time()
files = sorted([root+'/'+files for root,_,f in os.walk(main_data_path)  for files in f ])
total_scores_all = Parallel(n_jobs=20)(delayed(parallel_SRNN)(path) for path in files)
print(time()-t)

In [ ]:
res = pd.concat([pd.DataFrame(total_scores_all[i]) for i in range(len(total_scores_all))], axis=0)
res.to_csv('srnn.csv')
# res.groupby('dataset').mean()

### Parallel LIS-RNN

In [ ]:
def parallel_LISRNN(path): 
    dir_ = path.split('/')[-2]
    if dir_ == 'yahoo' or dir_ == 'power':
        data_df = pd.read_csv(path)[['value']]
        label_df = pd.read_csv(path).iloc[:, -1]

    else:
        data_df = pd.read_csv(path).iloc[:, 0:-1].astype(float)   
        label_df = pd.read_csv(path).iloc[:, -1].astype(int)    


    if dir_ == 'ECG':
        return process_LISRNN(path.split('/')[-1],data_df,label_df, seq_length = 32, stride = 16)
    else:
        return process_LISRNN(path.split('/')[-1],data_df,label_df, seq_length = 8, stride = 4)


In [ ]:
t = time()
files = sorted([root+'/'+files for root,_,f in os.walk(main_data_path)  for files in f ])
total_scores_all = Parallel(n_jobs=20)(delayed(parallel_LISRNN)(path) for path in files)
print(time()-t)

In [ ]:
res = pd.concat([pd.DataFrame(total_scores_all[i]) for i in range(len(total_scores_all))], axis=0)
res.to_csv('lisrnn.csv')
# res.groupby('dataset').mean()

#Friedman Test & Post Hoc Evaluation

In [ ]:
!pip install scikit-posthocs

In [ ]:
from google.colab import drive
import pandas as pd
from scipy import stats
import scikit_posthocs as sp
import numpy as np

In [ ]:
drive.mount('/content/drive', force_remount=True)
results_path = "/content/drive/MyDrive/ML_Final_Project/Results_final/"

In [ ]:
# get relevant data from result file
def get_result_df(filename):
  result_df = pd.read_csv(results_path + filename)[['dataset','f1','pr_auc','roc_auc','tpr','fpr','precision','accuracy']]
  return result_df.groupby('dataset').mean()

In [ ]:
result_lstm_df = get_result_df("lstm.csv")
result_srnn_df = get_result_df("srnn.csv")
result_lisrnn_df = get_result_df("lisrnn.csv")

In [ ]:
result_lstm_df

In [ ]:
result_srnn_df

In [ ]:
result_lisrnn_df

In [ ]:
# perform the Friedman test on all of the metrics
print("tpr: ", stats.friedmanchisquare(result_lstm_df['tpr'], result_srnn_df['tpr'], result_lisrnn_df['tpr']))
print("fpr: ", stats.friedmanchisquare(result_lstm_df['fpr'], result_srnn_df['fpr'], result_lisrnn_df['fpr']))
print("precision: ", stats.friedmanchisquare(result_lstm_df['precision'], result_srnn_df['precision'], result_lisrnn_df['precision']))
print("accuracy: ", stats.friedmanchisquare(result_lstm_df['accuracy'], result_srnn_df['accuracy'], result_lisrnn_df['accuracy']))
print("roc_auc: ", stats.friedmanchisquare(result_lstm_df['roc_auc'], result_srnn_df['roc_auc'], result_lisrnn_df['roc_auc']))
print("pr_auc: ", stats.friedmanchisquare(result_lstm_df['pr_auc'], result_srnn_df['pr_auc'], result_lisrnn_df['pr_auc']))

### Accuracy Post-hoc

In [ ]:
lstm_accuracy = result_lstm_df['accuracy'].values
srnn_accuracy = result_srnn_df['accuracy'].values
lisrn_accuracy = result_lisrnn_df['accuracy'].values

data = np.array([lstm_accuracy, srnn_accuracy, lisrn_accuracy])

sp.posthoc_nemenyi_friedman(data.T)

### Tpr Post-hoc

In [ ]:
lstm_tpr = result_lstm_df['tpr'].values
srnn_tpr = result_srnn_df['tpr'].values
lisrn_tpr = result_lisrnn_df['tpr'].values

data = np.array([lstm_tpr, srnn_tpr, lisrn_tpr])

sp.posthoc_nemenyi_friedman(data.T)

### Fpr Post-hoc

In [ ]:
lstm_fpr = result_lstm_df['fpr'].values
srnn_fpr = result_srnn_df['fpr'].values
lisrn_fpr = result_lisrnn_df['fpr'].values

data = np.array([lstm_fpr, srnn_fpr, lisrn_fpr])

sp.posthoc_nemenyi_friedman(data.T)